# 문제 정의
- 환자의 당뇨병 여부를 예측하시오.
 - 제공된 데이터 목록: diabetes_train.csv, diabetes_test.csv
 - 예측할 컬럼: Outcome(0: 정상, 1: 당뇨병)
- 학습용 데이터(train)를 이용해 환자의 당뇨병을 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측값을 다음과 같은 형식의 CSV 파일로 생성하시오.
 - 제출 파일은 다음 1개의 컬럼을 포함해야 한다.
  - pred: 예측값(당뇨병일 확률)
  - 제출 파일명: 'result.csv'
 - 제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점한다.

In [ ]:
# 데이터 업로드
from google.colab import files
uploads = files.upload()

Saving diabetes_train.csv to diabetes_train.csv
Saving diabetes_test.csv to diabetes_test.csv


In [ ]:
# 데이터 불러오기
import pandas as pd
train = pd.read_csv('diabetes_train.csv')
test = pd.read_csv('diabetes_test.csv')

In [ ]:
# 데이터 크기 확인
train.shape, test.shape

((614, 9), (154, 8))

In [ ]:
# object col 확인
train.info() # 인코딩 불필요(모두 수치형)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               614 non-null    int64  
 1   Glucose                   614 non-null    int64  
 2   BloodPressure             614 non-null    int64  
 3   SkinThickness             614 non-null    int64  
 4   Insulin                   614 non-null    int64  
 5   BMI                       614 non-null    float64
 6   DiabetesPedigreeFunction  614 non-null    float64
 7   Age                       614 non-null    int64  
 8   Outcome                   614 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 43.3 KB


In [ ]:
# 기술통계량 확인
train.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,3.789902,120.659609,69.366450,20.969055,80.382736,32.067427,0.475490,33.250814,0.343648
std,3.384519,32.556686,18.847224,15.923677,111.332249,7.888971,0.326519,11.789160,0.475313
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.500000,0.000000,0.000000,27.300000,0.245500,24.000000,0.000000
50%,3.000000,116.000000,72.000000,23.000000,36.000000,32.100000,0.381500,29.000000,0.000000
75%,6.000000,140.750000,80.000000,32.000000,128.750000,36.600000,0.629750,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,680.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
# 결측치 확인
train.isnull().sum() # 결측치 없음

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [ ]:
# 기본적인 이상치 중 말 안되는 것은 사실상 결측치 (혈당 0, 혈압 0, BMI 0, SkinThickness 0, Insulin 0)
import numpy as np
train[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = train[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.nan)
test[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = test[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.nan)
train.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,614.000000,609.000000,589.000000,441.000000,320.000000,606.000000,614.000000,614.000000,614.000000
mean,3.789902,121.650246,72.310696,29.195011,154.234375,32.490759,0.475490,33.250814,0.343648
std,3.384519,30.788714,12.532281,10.609304,111.320197,7.020093,0.326519,11.789160,0.475313
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,100.000000,64.000000,22.000000,78.750000,27.400000,0.245500,24.000000,0.000000
50%,3.000000,116.000000,72.000000,29.000000,125.000000,32.400000,0.381500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,192.250000,36.750000,0.629750,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,680.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
# 결측치 대치
cols = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
for col in cols:
  train[col].fillna(train[col].median(), inplace =True)
  test[col].fillna(train[col].median(), inplace = True)
train.describe()

<ipython-input-9-d5d42c6e309b>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna(train[col].median(), inplace =True)
<ipython-input-9-d5d42c6e309b>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,3.789902,121.604235,72.298046,29.140065,140.236156,32.489577,0.475490,33.250814,0.343648
std,3.384519,30.667102,12.274222,8.988842,81.623622,6.974142,0.326519,11.789160,0.475313
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,100.000000,64.000000,25.000000,120.000000,27.500000,0.245500,24.000000,0.000000
50%,3.000000,116.000000,72.000000,29.000000,125.000000,32.400000,0.381500,29.000000,0.000000
75%,6.000000,140.750000,80.000000,32.000000,128.750000,36.600000,0.629750,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,680.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
# 전처리 시작
# target 분리
target = train.pop('Outcome')

In [ ]:
# test val 분리
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    train,
    target,
    test_size = 0.2,
    random_state = 123
)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((491, 8), (123, 8), (491,), (123,))

In [ ]:
# Randomforest 기준 학습
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 123)
rf.fit(X_train, y_train)
pred_rf = rf.predict_proba(X_val)

In [ ]:
# roc_auc확인
from sklearn.metrics import roc_auc_score
roc_auc_rf = roc_auc_score(y_val, pred_rf[:,1])

In [ ]:
# lightgbm
import lightgbm as lgb
lgbmc = lgb.LGBMClassifier(random_state = 123,verbose=-1)
lgbmc.fit(X_train, y_train)

pred_lgbmc = lgbmc.predict_proba(X_val)

In [ ]:
#roc_auc - lightgbm
roc_auc_lgbmc = roc_auc_score(y_val, pred_lgbmc[:,1])

In [ ]:
roc_auc_rf, roc_auc_lgbmc

(np.float64(0.8531944444444444), np.float64(0.8583333333333333))

# 성능 개선

In [ ]:
# 스케일링
from sklearn.preprocessing import StandardScaler
scaler_st = StandardScaler()
from sklearn.preprocessing import MinMaxScaler
scaler_mm = MinMaxScaler()
from sklearn.preprocessing import RobustScaler
scaler_rb = RobustScaler()

In [ ]:
# 스케일링 진행
#standard scaler
train_st = scaler_st.fit_transform(train)
test_st = scaler_st.transform(test)
#mimnax scaler
train_mm = scaler_mm.fit_transform(train)
test_mm = scaler_mm.transform(test)
#robust scaler
train_rb = scaler_rb.fit_transform(train)
test_rb = scaler_rb.transform(test)

In [ ]:
# 데이터 분할
from sklearn.model_selection import train_test_split
X_train_st, X_val_st, y_train_st, y_val_st = train_test_split(
    train_st,
    target,
    test_size = 0.2,
    random_state = 123
)

X_train_mm, X_val_mm, y_train_mm, y_val_mm = train_test_split(
    train_mm,
    target,
    test_size = 0.2,
    random_state = 123
)

X_train_rb, X_val_rb, y_train_rb, y_val_rb = train_test_split(
    train_rb,
    target,
    test_size = 0.2,
    random_state = 123
)

In [ ]:
# 머신러닝 학습
import lightgbm as lgb
lgbmc = lgb.LGBMClassifier(random_state=123, verbose= -1)

#standard
lgbmc.fit(X_train_st, y_train_st)

pred_st=lgbmc.predict_proba(X_val_st)

roc_auc_st = roc_auc_score(y_val_st, pred_st[:,1])
roc_auc_st

#minmax
lgbmc.fit(X_train_mm, y_train_mm)

pred_mm=lgbmc.predict_proba(X_val_mm)

roc_auc_mm = roc_auc_score(y_val_mm, pred_mm[:,1])
roc_auc_mm

#robust

lgbmc.fit(X_train_rb,y_train_rb)
pred_rb=lgbmc.predict_proba(X_val_st)
roc_auc_rb = roc_auc_score(y_val_rb, pred_rb[:,1])
print("roc_auc_st:",roc_auc_st,"roc_auc_mm:",roc_auc_mm,"roc_auc_rb:",roc_auc_rb)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


roc_auc_st: 0.8725 roc_auc_mm: 0.8583333333333333 roc_auc_rb: 0.8683333333333334


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
# roc_auc_st 로 결정
final_model = lgb.LGBMClassifier(random_state=123, verbose=-1)
final_model.fit(X_train_st, y_train_st)
pred = lgbmc.predict_proba(test_st)
submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv('result.csv',index=False)
df = pd.read_csv('result.csv')
df

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


,pred
0,0.003036
1,0.013365
2,0.012397
3,0.000668
4,0.000724
...,...
149,0.660200
150,0.002676
151,0.000169
152,0.801518
